# **Regressão Linear Múltipla — Módulo 4, Notebook 1/4**

---

## Índice

1. [Introdução ao Problema](#introducao)
2. [Intuição](#intuicao)
3. [Compreendendo o Algoritmo](#como-funciona)
4. [Formulação Formal](#formal)
5. [Métricas de Avaliação](#metricas)
6. [Implementação do Zero](#from-scratch)
7. [Usando Scikit-learn](#sklearn)

---

<a id='introducao'></a>
## **Introdução ao Problema**

Imagine que você é um corretor de imóveis iniciante e sua gerente, a Sra. Ana, uma avaliadora experiente, consegue estimar o preço de uma casa apenas olhando para algumas de suas características. Você, por outro lado, se sente um pouco perdido. Como ela faz isso?

Você pergunta a ela, e a Sra. Ana responde: "É uma questão de entender o impacto de cada característica. Um quarto a mais não adiciona um valor fixo; o impacto dele depende da área total da casa. A localização, o número de banheiros, a idade do imóvel... tudo isso contribui para o preço final, e meu trabalho é ponderar a importância de cada um desses fatores."

E como podemos criar um modelo que aprenda essas ponderações a partir de dados históricos? E se, em vez de depender apenas da intuição, pudéssemos usar a matemática para encontrar a 'receita' exata que combina essas características para chegar ao preço mais provável?

É exatamente esse o problema que a **Regressão Linear Múltipla** se propõe a resolver. Ela nos ajuda a entender e quantificar a relação entre **duas ou mais variáveis independentes** (nossos 'ingredientes', como área, quartos, etc.) e uma **variável dependente** contínua (nosso 'resultado', o preço da casa).

**Diferença importante:** Nos algoritmos anteriores (KNN, SVM, etc.), trabalhamos com **classificação** - prever categorias discretas (ex: "Setosa", "Versicolor", "Virginica" ou "Sobreviveu/Não Sobreviveu"). Agora, estamos lidando com **regressão** - prever valores contínuos (ex: preço de R$ 250.000, R$ 380.000, etc.).

<a id='intuicao'></a>
## **Intuição**

A Regressão Linear Múltipla expande a ideia da regressão linear simples. Se na regressão simples tentamos encontrar a **melhor linha** que descreve a relação entre *uma* variável de entrada e a saída, na regressão múltipla, tentamos encontrar o **melhor 'plano' ou 'hiperplano'** que descreve a relação entre *múltiplas* variáveis de entrada e a saída.

Pense na 'receita de bolo' da Sra. Ana para precificar uma casa. A receita dela pode ser algo como:

**Preço Estimado = (Preço Base) + (Valor por m² × Área) + (Valor por Quarto × N° de Quartos) - (Depreciação por Ano × Idade do Imóvel)**

O que a Regressão Linear Múltipla faz é exatamente encontrar os valores para 'Preço Base', 'Valor por m²', 'Valor por Quarto' e 'Depreciação por Ano'. Esses valores são os **coeficientes** (ou pesos) do nosso modelo. Cada coeficiente nos diz o seguinte:

> *Mantendo todas as outras características constantes, qual é o impacto médio de se adicionar uma unidade desta característica no resultado final?*

Por exemplo, se o 'Valor por Quarto' for R$ 50.000, isso significa que, para casas com a mesma área e idade, adicionar um quarto extra tende a aumentar o preço em R$ 50.000, em média.

Desta vez, vamos trabalhar com o dataset **[California Housing](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset)** - dados reais de preços de casas na Califórnia. Queremos prever o preço médio de uma casa com base em características como:

- **MedInc**: Renda média da região
- **HouseAge**: Idade média das casas
- **AveRooms**: Número médio de cômodos
- **AveBedrms**: Número médio de quartos
- **Population**: População da região
- **AveOccup**: Média de ocupantes por casa
- **Latitude** e **Longitude**: Localização geográfica

Pergunta: **Qual será o preço de uma casa com essas características específicas?**

In [ ]:
# =======================================================
# IMPORTANDO AS BIBLIOTECAS
# =======================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split


In [ ]:
# CARREGANDO O CALIFORNIA HOUSING DATASET
housing = fetch_california_housing()

X = housing.data  # Features: 8 características das casas
y = housing.target  # Target: Preço médio (em centenas de milhares de dólares)

print("Dataset California Housing:")
print(f"Número de amostras: {X.shape[0]}")
print(f"Número de features: {X.shape[1]}")
print("=" * 50)

print("\nFeatures:")
for i, nome in enumerate(housing.feature_names):
    print(f"{i+1}. {nome}")
print("=" * 50)

print("\nVisualizando os dados:")
df = pd.DataFrame(X, columns=housing.feature_names)
df['Price'] = y
print(df.head())
print("\nEstatísticas descritivas:")
print(df.describe())

In [ ]:
# Analisando correlação com o preço
print("Correlação de cada feature com o Preço:")
print("=" * 50)

correlacoes = []
for i, feature_name in enumerate(housing.feature_names):
    corr = np.corrcoef(X[:, i], y)[0, 1]
    correlacoes.append((feature_name, corr))
    print(f"{feature_name:15} → Correlação: {corr:+.4f}")

print("\n" + "=" * 50)
print("Selecionando as 3 features com maior correlação:")
correlacoes_sorted = sorted(correlacoes, key=lambda x: abs(x[1]), reverse=True)
features_selecionadas = correlacoes_sorted[:3]

print()
for i, (feat_name, corr_value) in enumerate(features_selecionadas, 1):
    print(f"{i}. {feat_name:15} → {abs(corr_value):.4f} (impacto forte no preço)")

# Vou simplificar para as 3 features com maior correlação
features = [feat[0] for feat in features_selecionadas]
indices = [list(housing.feature_names).index(f) for f in features]

print(f"\nFeatures selecionadas: {features}")
print(f"Índices: {indices}")

X_simples = X[:, indices]

In [ ]:
# Criando um exemplo: uma casa específica para prever o preço
nova_casa = np.array([5.0, 6.0, 15.0])  # MedInc=5.0, AveRooms=6.0, HouseAge=15.0

print("Nossa casa para avaliação:")
print(f"  Renda média da região: ${nova_casa[0] * 10000:.0f}")
print(f"  Número médio de cômodos: {nova_casa[1]:.1f}")
print(f"  Idade média das casas: {nova_casa[2]:.0f} anos")
print("\nQual será o preço estimado?")

In [ ]:
# Visualizando a relação entre features e preço
fig, axes = plt.subplots(1, 3, figsize=(18, 4))
fig.suptitle("Relação entre Features e Preço das Casas", fontsize=16, fontweight='bold')

for i, (ax, feature) in enumerate(zip(axes, features)):
    # Amostragem para melhor visualização
    sample_indices = np.random.choice(X.shape[0], 1000, replace=False)
    
    ax.scatter(X_simples[sample_indices, i], y[sample_indices], 
               alpha=0.5, s=20, c=y[sample_indices], cmap='viridis')
    ax.set_xlabel(feature, fontsize=12)
    ax.set_ylabel("Preço (x$100k)", fontsize=12)
    ax.set_title(f"{feature} vs Preço")
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

<a id='como-funciona'></a>
## **Compreendendo o Algoritmo**

### **1. A Equação Linear**

O modelo cria uma equação que combina todas as features:

$$\text{Preço} = \beta_0 + \beta_1 \times \text{MedInc} + \beta_2 \times \text{AveRooms} + \beta_3 \times \text{HouseAge}$$

Onde:
- $\beta_0$ é o **intercepto** (preço base)
- $\beta_1, \beta_2, \beta_3$ são os **coeficientes** (peso de cada feature)

### **2. Encontrando os Melhores Coeficientes**

O algoritmo tenta encontrar os valores de $\beta$ que minimizam o **erro quadrático médio** (MSE):

$$\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

Onde:
- $y_i$ é o preço real
- $\hat{y}_i$ é o preço previsto
- $n$ é o número de amostras

### **3. Interpretação dos Coeficientes**

Cada coeficiente nos diz: "Se eu aumentar essa feature em 1 unidade (mantendo todas as outras constantes), quanto o preço vai mudar?"

- Se $\beta_1 = 40.000$: Aumentar a renda média em 1 unidade aumenta o preço em $40k
- Se $\beta_3 = -2.000$: Cada ano a mais de idade diminui o preço em $2k

In [ ]:
# Vamos treinar um modelo simples para ver os coeficientes
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(
    X_simples, y, test_size=0.2, random_state=42
)

# Treinar o modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Coeficientes
print("Coeficientes do modelo:")
print(f"Intercepto (β₀): ${modelo.intercept_ * 100000:.2f}")
print()
for nome, coef in zip(features, modelo.coef_):
    print(f"  {nome} (β): ${coef * 100000:.2f}")
    print(f"    - Aumentar {nome} em 1 unidade muda o preço em ${abs(coef * 100000):.2f}")
    print()

# Fazer previsão para nossa casa
preco_previsto = modelo.predict([nova_casa])[0]
print("=" * 60)
print(f"Preço estimado para nossa casa: ${preco_previsto * 100000:.2f}")
print("=" * 60)



**Desafio: Parece contraintuitivo que aumentar a quantidade de quartos diminua o preço da casa, levante hipóteses do porque isso está acontecendo**

**Resposta e Análise:**

Este é um ótimo exemplo de **multicolinearidade**! Aqui estão as possíveis causas:

1. **Multicolinearidade entre MedInc e AveRooms:**
   - Em regiões de alta renda, casas têm mais cômodos
   - Em regiões de baixa renda, casas têm menos cômodos
   - O modelo tem dificuldade em separar qual é realmente o efeito

2. **Captura de correlações espúrias:**
   - AveRooms está **inversamente correlacionado com densidade** (casas menores têm menos quartos)
   - Regiões com casas menores (menos quartos) podem ter outros fatores que aumentam o preço (proximidade do centro)

3. **Interpretação errada dos coeficientes:**
   - O coeficiente de AveRooms não significa "adicionar um quarto sempre diminui o preço"
   - Significa: "Mantendo MedInc e HouseAge constantes, adicionar quartos está associado a redução de preço"
   - Mas na realidade, se você adiciona um quarto, a renda e idade também mudam!

**Para confirmar:**

In [ ]:
# Analisando a multicolinearidade
print("ANÁLISE DE MULTICOLINEARIDADE")
print("=" * 60)
print("\n1. Matriz de Correlação entre as features selecionadas:")
print()

# Criar dataframe com as features simplificadas
df_features = pd.DataFrame(X_simples, columns=features)
df_features['Price'] = y

print(df_features.corr().round(3))

print("\n2. Correlação entre as features independentes:")
print()
for i in range(len(features)):
    for j in range(i+1, len(features)):
        corr = np.corrcoef(X_simples[:, i], X_simples[:, j])[0, 1]
        print(f"   {features[i]} vs {features[j]}: {corr:+.4f}")

print("\n3. Interpretação:")
if abs(np.corrcoef(X_simples[:, 0], X_simples[:, 1])[0, 1]) > 0.5:
    print(f"    MedInc e AveRooms têm correlação forte!")
    print(f"   Isso explica por que o coeficiente de AveRooms é negativo!")
    print(f"   Quando MedInc é mantido constante, AveRooms acaba capturando")
    print(f"   a relação inversa com densidade (menores casas = mais pobreza).")
else:
    print(f"   As features têm correlação fraca. Coeficientes são confiáveis.")

In [ ]:
# Visualização: Valores reais vs Previstos
y_pred = modelo.predict(X_test)

plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5, s=20)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Previsão Perfeita')
plt.xlabel("Preço Real (x$100k)", fontsize=12)
plt.ylabel("Preço Previsto (x$100k)", fontsize=12)
plt.title("Preços Reais vs Previstos - Regressão Linear Múltipla", 
          fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Calcular métricas de desempenho
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("=" * 70)
print("MÉTRICAS DE DESEMPENHO DO MODELO")
print("=" * 70)
print(f"  MSE (Erro Quadrático Médio):      {mse:.6f}")
print(f"  RMSE (Raiz do MSE):               {rmse:.6f} (≈ ${rmse * 100000:,.0f})")
print(f"  MAE (Erro Absoluto Médio):        {mae:.6f} (≈ ${mae * 100000:,.0f})")
print(f"  R² (Coeficiente de Determinação): {r2:.6f}")
print("=" * 70)

print("\nINTERPRETAÇÃO DOS RESULTADOS:\n")

print(f"1. **RMSE (≈ ${rmse * 100000:,.0f})**")
print(f"   → Em média, nossas previsões erraram por ${rmse * 100000:,.0f}")
print(f"   → Para uma casa de preço médio (~$200k), isso é um erro de ~{(rmse*100000/200000)*100:.1f}%")
print(f"   → Razoável para um modelo linear com apenas 3 features\n")

print(f"2. **MAE (≈ ${mae * 100000:,.0f})**")
print(f"   → Erro absoluto: em média erramos por ${mae * 100000:,.0f} (sem penalizar outliers)")
print(f"   → Comparação: MAE < RMSE significa que não há outliers extremos\n")

print(f"3. **R² = {r2:.4f} ({r2*100:.2f}%)**")
if r2 >= 0.7:
    print(f"   EXCELENTE! O modelo explica {r2*100:.1f}% da variação nos preços")
    print(f"   - Significa: 70%+ da variação é capturada pelas 3 features escolhidas")
elif r2 >= 0.5:
    print(f"    BOM! O modelo explica {r2*100:.1f}% da variação nos preços")
    print(f"   - Há ainda {(1-r2)*100:.1f}% de variação inexplicada (outras features, randomness)")
elif r2 >= 0.3:
    print(f"    MODERADO. Explicamos {r2*100:.1f}% da variação")
    print(f"   - Modelo não é ótimo, mas melhor que adivinhar a média")
else:
    print(f"   FRACO. Explicamos apenas {r2*100:.1f}% da variação")
    print(f"   - Outras features/relações não-lineares são importantes")

print(f"\n4. **Conclusão Prática:**")
print(f"   - Com 3 features simples, nosso modelo é {r2*100:.1f}% confiável")
print(f"   - Se quisermos melhorar: usar mais features, relações não-lineares, ou")
print(f"     regularização (Ridge/Lasso) para lidar com multicolinearidade")

<a id='formal'></a>
## **Formulação Formal**

**Modelo Matemático**

Dado um conjunto de dados com $n$ amostras e $p$ features, a regressão linear múltipla modela a relação entre as variáveis independentes $X$ e a variável dependente $y$ através da equação:

$$y_i = \beta_0 + \beta_1 x_{i1} + \beta_2 x_{i2} + ... + \beta_p x_{ip} + \epsilon_i$$

Ou em forma vetorial:

$$y = X\beta + \epsilon$$

Onde:
- $y$ é o vetor de valores observados (target)
- $X$ é a matriz de features ($n \times p$)
- $\beta$ é o vetor de coeficientes ($p \times 1$)
- $\epsilon$ é o vetor de erros (resíduos)

**Objetivo: Método dos Mínimos Quadrados**

Encontrar $\beta$ que minimize a soma dos quadrados dos resíduos (SSR):

$$\text{SSR} = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 = \sum_{i=1}^{n} (y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij})^2$$

Ou em forma matricial:

$$\text{SSR} = (y - X\beta)^T(y - X\beta)$$

**Solução Analítica (Equação Normal)**

A solução que minimiza SSR é:

$$\hat{\beta} = (X^TX)^{-1}X^Ty$$

Esta é a **Equação Normal** - a solução de forma fechada para regressão linear.

**Premissas do Modelo**

1. **Linearidade**: A relação entre X e y é linear
2. **Independência**: As observações são independentes
3. **Homocedasticidade**: A variância dos erros é constante
4. **Normalidade**: Os erros seguem distribuição normal
5. **Não-colinearidade**: As features não são altamente correlacionadas entre si

<a id='metricas'></a>
## **Métricas de Avaliação**

Para problemas de regressão, usamos métricas diferentes das de classificação:

### **1. MSE (Mean Squared Error)**

$$\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

Penaliza erros grandes mais fortemente (devido ao quadrado).

### **2. RMSE (Root Mean Squared Error)**

$$\text{RMSE} = \sqrt{\text{MSE}}$$

Tem a mesma unidade que a variável target (mais interpretável).

### **3. MAE (Mean Absolute Error)**

$$\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$$

Menos sensível a outliers que o MSE.

### **4. R² (Coeficiente de Determinação)**

$$R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$$

Indica a proporção da variância explicada pelo modelo (0 a 1).
- $R^2 = 1$: Modelo perfeito
- $R^2 = 0$: Modelo não melhor que a média

<a id='from-scratch'></a>
## **Implementação do Zero**

Agora que você entende como funciona a Regressão Linear Múltipla, tente implementar o algoritmo do zero, usando apenas NumPy!

In [ ]:
class RegressaoLinearMultipla:
    def __init__(self):
        """
        Inicializa o modelo de Regressão Linear Múltipla
        """
        self.coef_ = None  # Coeficientes β
        self.intercept_ = None  # Intercepto β₀
    
    def fit(self, X, y):
        """
        Treina o modelo usando a Equação Normal
        
        Parâmetros:
        X (array-like): Matriz de features (n_samples, n_features)
        y (array-like): Vetor target (n_samples,)
        """
        # Adicionar coluna de 1s para o intercepto
        X_b = np.c_[np.ones((X.shape[0], 1)), X]  # [1, x1, x2, ..., xp]
        
        # Aplicar a Equação Normal: β = (X^T X)^(-1) X^T y
        beta = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
        
        # Separar intercepto dos coeficientes
        self.intercept_ = beta[0]
        self.coef_ = beta[1:]
        
        return self
    
    def predict(self, X):
        """
        Faz previsões para novos dados
        
        Parâmetros:
        X (array-like): Matriz de features (n_samples, n_features)
        
        Retorna:
        array: Previsões (n_samples,)
        """
        # y_pred = β₀ + β₁x₁ + β₂x₂ + ... + βₚxₚ
        return X.dot(self.coef_) + self.intercept_
    
    def score(self, X, y):
        """
        Calcula o R² (coeficiente de determinação)
        
        Parâmetros:
        X (array-like): Matriz de features
        y (array-like): Vetor target verdadeiro
        
        Retorna:
        float: R² score
        """
        y_pred = self.predict(X)
        
        # R² = 1 - (SS_res / SS_tot)
        ss_res = np.sum((y - y_pred) ** 2)  # Soma dos quadrados dos resíduos
        ss_tot = np.sum((y - np.mean(y)) ** 2)  # Soma total dos quadrados
        
        r2 = 1 - (ss_res / ss_tot)
        return r2
    
    def mean_squared_error(self, y_true, y_pred):
        """
        Calcula o MSE
        
        Parâmetros:
        y_true (array-like): Valores verdadeiros
        y_pred (array-like): Valores previstos
        
        Retorna:
        float: MSE
        """
        # MSE = (1/n) Σ(y_i - ŷ_i)²
        mse = np.mean((y_true - y_pred) ** 2)
        return mse

In [ ]:
# Agora vamos testar nossa implementação com os dados do California Housing
# (já carregados nas células anteriores)

print("=" * 60)
print("TESTANDO NOSSA IMPLEMENTAÇÃO")
print("=" * 60)

# Dividindo os dados (já temos X_simples e y)
X_train, X_test, y_train, y_test = train_test_split(
    X_simples, y, test_size=0.2, random_state=42
)

print(f"\nDados preparados!")
print(f"Tamanho do treino: {X_train.shape[0]} amostras")
print(f"Tamanho do teste: {X_test.shape[0]} amostras")
print(f"Número de features: {X_train.shape[1]}")
print(f"Features: {features}")

<a id='sklearn'></a>
## **Usando Scikit-learn e Comparando com Nossa Implementação**

Agora vamos usar a implementação pronta do scikit-learn e comparar com nossa implementação do zero!

In [ ]:
# Testando nossa implementação
print("=" * 70)
print("TESTANDO NOSSA IMPLEMENTAÇÃO (do zero com NumPy)")
print("=" * 70)

modelo_nosso = RegressaoLinearMultipla()
modelo_nosso.fit(X_train, y_train)

print("\nCoeficientes da nossa implementação:")
print(f"  Intercepto: ${modelo_nosso.intercept_ * 100000:.2f}")
for feat, coef in zip(features, modelo_nosso.coef_):
    print(f"  {feat}: ${coef * 100000:.2f}")

# Previsões
y_pred_nosso = modelo_nosso.predict(X_test)
r2_nosso = modelo_nosso.score(X_test, y_test)
rmse_nosso = np.sqrt(modelo_nosso.mean_squared_error(y_test, y_pred_nosso))

print(f"\nR² (nossa impl): {r2_nosso:.6f}")
print(f"RMSE (nossa impl): ${rmse_nosso * 100000:,.0f}")

# Comparando com scikit-learn
print("\n" + "=" * 70)
print("COMPARANDO COM SCIKIT-LEARN")
print("=" * 70)

modelo_sklearn = LinearRegression()
modelo_sklearn.fit(X_train, y_train)

print("\nCoeficientes do scikit-learn:")
print(f"  Intercepto: ${modelo_sklearn.intercept_ * 100000:.2f}")
for feat, coef in zip(features, modelo_sklearn.coef_):
    print(f"  {feat}: ${coef * 100000:.2f}")

y_pred_sklearn = modelo_sklearn.predict(X_test)
r2_sklearn = modelo_sklearn.score(X_test, y_test)
rmse_sklearn = np.sqrt(mean_squared_error(y_test, y_pred_sklearn))

print(f"\nR² (scikit-learn): {r2_sklearn:.6f}")
print(f"RMSE (scikit-learn): ${rmse_sklearn * 100000:,.0f}")

# Comparação
print("\n" + "=" * 70)
print("COMPARAÇÃO: Nossa Implementação vs Scikit-learn")
print("=" * 70)

print(f"\nDiferença no R²:        {abs(r2_nosso - r2_sklearn):.9f} (praticamente zero!)")
print(f"Diferença no RMSE:      ${abs(rmse_nosso - rmse_sklearn) * 100000:.2f}")
print(f"\nDiferença de coeficientes:")
for feat, coef_nosso, coef_sk in zip(features, modelo_nosso.coef_, modelo_sklearn.coef_):
    diff = abs(coef_nosso - coef_sk)
    print(f"  {feat}: ${diff * 100000:.6f}")

print(f"\nDiferença de intercepto: ${abs(modelo_nosso.intercept_ - modelo_sklearn.intercept_) * 100000:.6f}")

print("\nCONCLUSÃO:")
print("   Nossa implementação está consistente.")
print("   Os resultados são equivalentes aos do scikit-learn (diferenças muito pequenas).")
print("   Agora você tem segurança na matemática por trás da Regressão Linear Múltipla.")

---

**Fechando a conversa:**
- Regressão linear múltipla é um bom primeiro passo quando você quer explicar o impacto de cada variável.
- Funciona melhor quando a relação é aproximadamente linear e as features não brigam entre si (pouca multicolinearidade).
- Se os erros parecerem grandes ou os coeficientes estiverem esquisitos, teste alternativas como SVR, Árvores ou Ensembles.
- Antes de trocar de modelo, tente padronizar as features e usar regularização (Ridge/Lasso) para estabilizar os coeficientes.


---
<-- [**Anterior: Módulo 3 - Classificação**](../03_Classificacao/06_ensemble_classificacao.ipynb) | [**Próximo: SVR**](02_svr.ipynb) -->